# AIMMS Tutorial Example

The following problem is a [tutorial example taken from AIMMS](https://download.aimms.com/aimms/download/references/AIMMS_tutorial_beginner.pdf). It demonstrates the capabilities of Pyomo and how to construct a GUI using `molaqt` to solve the problem.

# Problem Statement

Beer is shipped in lorries from two brewing plants (Haarlem, Eindhoven) to five customers (Amsterdam, Breda, Gouda, Amersfoort, Den Bosch) in a fixed time period. The objective is to minimise the cost of moving the beer such that the customer demand is met and shipments do not exceed the
available supply from each brewery. 

The available supply at each plant and the required demand by each customer (measured in lorry loads) is shown in the table below. The cost associated with moving one lorry load from a plant to a
customer is shown as a grid. .

| | **Amsterdam** | **Breda** | **Gouda** | **Amersfoort** | **Den Bosch** | Supply |
| --- | --- | --- | --- | --- | --- | --- |
| **Haarlem** | 131 | 405 | 188 | 396 | 485 | 47 |
| **Eindhoven** | 554 | 351 | 479 | 366 | 155 | 63 |
| Demand | 28 | 16 | 22 | 31 | 12 |

# Concrete Pyomo Implementation

We build a concrete optimisation model using pyomo to solve this problem.

In [1]:
from pyomo.environ import *
model = ConcreteModel()

We adopt the following indices

* $p\in P$ plants
* $c\in C$ customers

and implement them in the model using `Sets`.

In [2]:
model.P = Set(initialize=['Haarlem', 'Eindhoven'])
model.C = Set(initialize=['Amsterdam', 'Breda', 'Gouda', 'Amersfoort', 'Den Bosch'])

The model parameters are for supply, demand and unit cost:

* $S_p$ supply at plant $p$
* $D_c$ demand by customer $c$
* $U_{p,c}$ unit transport cost from $p$ to $c$

In Pyomo these are implemented using `Params`. To get the unit cost data into the model we use Pandas and convert a `DataFrame` to a `dict` indexed by (plant, customer).

In [3]:
import pandas as pd
s_data = {'Haarlem': 47, 'Eindhoven': 63}
model.S = Param(model.P, initialize=s_data)
d_data = {'Amsterdam': 28, 'Breda': 16, 'Gouda': 22, 'Amersfoort': 31, 'Den Bosch': 12}
model.D = Param(model.C, initialize=d_data)
cost = [[131, 405, 188, 396, 485], [554, 351, 479, 366, 155]]
cost_df = pd.DataFrame(cost, index=list(model.P), columns=list(model.C))
model.U = Param(model.P, model.C, initialize=cost_df.stack().to_dict())
model.U.pprint()

U : Size=10, Index=U_index, Domain=Any, Default=None, Mutable=False
    Key                         : Value
    ('Eindhoven', 'Amersfoort') :   366
     ('Eindhoven', 'Amsterdam') :   554
         ('Eindhoven', 'Breda') :   351
     ('Eindhoven', 'Den Bosch') :   155
         ('Eindhoven', 'Gouda') :   479
      ('Haarlem', 'Amersfoort') :   396
       ('Haarlem', 'Amsterdam') :   131
           ('Haarlem', 'Breda') :   405
       ('Haarlem', 'Den Bosch') :   485
           ('Haarlem', 'Gouda') :   188


The objective is to minimise the transport cost

$$
z = \sum_{p,c} U_{p,c}x_{p,c}
$$

where $x_{p,c}$ is the transport from $p$ to $c$. We define the variable $x_{p, c}$ indexed by sets $P$ and $C$ below.

In [4]:
model.x = Var(model.P, model.C, domain=PositiveReals)

We specify the domain of $x$ as the positive reals in order to satisfy the positive constraint

$$
x_{p,c} \ge 0
$$

The objective is defined by

In [5]:
model.z = Objective(expr=sum(model.U[p, c] * model.x[p, c] 
             for p in model.P for c in model.C), sense=minimize)

The supply constraint is

$$
\sum_c x_{p,c} \leq S_p
$$

and it is implemented in pyomo using the following function:

In [6]:
def supply_rule(model, p):
    return sum([model.x[p, c] for c in model.C]) <= model.S[p]
model.supply_constraint = Constraint(model.P, rule=supply_rule)

The demand constraint is

$$
\sum_p x_{p,c} \geq D_c
$$

and it is implemented in pyomo using the following function:

In [7]:
def demand_rule(model, c):
    return sum([model.x[p, c] for p in model.P]) >= model.D[c]
model.demand_constraint = Constraint(model.C, rule=demand_rule)

# Solution

We specify a solver and then call it.

In [8]:
opt = SolverFactory("glpk")
results = opt.solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 27499.0
  Upper bound: 27499.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 11
  Number of nonzeros: 21
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.014623165130615234
# ----------------------------------------------------------
#   Solution Information
# ------------------------------

The optimal solution below agrees with the solution given in the AIMMS tutorial.

In [9]:
model.x.pprint()

x : Size=10, Index=x_index
    Key                         : Lower : Value : Upper : Fixed : Stale : Domain
    ('Eindhoven', 'Amersfoort') :     0 :  31.0 :  None : False : False : PositiveReals
     ('Eindhoven', 'Amsterdam') :     0 :   0.0 :  None : False : False : PositiveReals
         ('Eindhoven', 'Breda') :     0 :  16.0 :  None : False : False : PositiveReals
     ('Eindhoven', 'Den Bosch') :     0 :  12.0 :  None : False : False : PositiveReals
         ('Eindhoven', 'Gouda') :     0 :   3.0 :  None : False : False : PositiveReals
      ('Haarlem', 'Amersfoort') :     0 :   0.0 :  None : False : False : PositiveReals
       ('Haarlem', 'Amsterdam') :     0 :  28.0 :  None : False : False : PositiveReals
           ('Haarlem', 'Breda') :     0 :   0.0 :  None : False : False : PositiveReals
       ('Haarlem', 'Den Bosch') :     0 :   0.0 :  None : False : False : PositiveReals
           ('Haarlem', 'Gouda') :     0 :  19.0 :  None : False : False : PositiveReals


# Mola Specification Class

In `mola` we define an abstract Pyomo model in a subclass of `Specification`.

In [10]:
from mola.specification5 import Specification
import pyomo.environ as pe
class AIMMSExampleSpecification(Specification):
    """
    A Mola Specification class containing an abstract model to solve the AIMMS simple tutorial problem.
    """
    user_defined_sets = {
        'P': 'Plants',
        'C': 'Customers',
    }
    user_defined_parameters = {
        'S': {'index': ['P'], 'doc': 'Supply available at plant p'},
        'D': {'index': ['C'], 'doc': 'Demand required by customer c'},
        'U': {'index': ['P', 'C'], 'doc': 'Cost per unit'},
    }
    # db parameters need to be constructed explicitly
    controllers = {"Standard": "StandardController"}
    default_settings = {
    }

    def __init__(self):

        # instance object to hold just the setting values
        self.settings = {k: v['value'] for k, v in self.default_settings.items()}

        # setup abstract model
        abstract_model = self.abstract_model = pe.AbstractModel()

        # user-defined sets
        for var, doc in self.user_defined_sets.items():
            abstract_model.add_component(var, pe.Set(doc=doc))

        # user-defined parameters
        for param, val in self.user_defined_parameters.items():
            idx = [abstract_model.component(i) for i in val['index']]
            abstract_model.add_component(param, pe.Param(*idx, doc=val['doc'], within=pe.Reals))

        # variables
        abstract_model.x = pe.Var(abstract_model.P, abstract_model.C,
                                     within=pe.NonNegativeReals, doc='Units')

        # objective
        def objective_rule(model):
            return sum(model.U[p, c] * model.x[p, c] for p in model.P for c in model.C)
        abstract_model.obj = pe.Objective(rule=objective_rule)

        # constraints
        def supply_rule(model, p):
            return sum([model.x[p, c] for c in model.C]) <= model.S[p]
        abstract_model.supply_constraint = Constraint(abstract_model.P, rule=supply_rule)

        def demand_rule(model, c):
            return sum([model.x[p, c] for p in model.P]) >= model.D[c]
        abstract_model.demand_constraint = Constraint(abstract_model.C, rule=demand_rule)

    def populate(self, json_files=None, elementary_flow_ref_ids=None, db_file=None):

        olca_dp = pyod.DataPortal()

        # user data
        for json_file in json_files:
            if json_file:
                olca_dp.load(filename=json_file)

        # use DataPortal to build concrete instance
        model_instance = self.abstract_model.create_instance(olca_dp)

        return model_instance

    def get_default_sets(self, d=None):
        user_sets = {
            'P': [],
            'C': [],
        }
        if d is not None:
            user_sets.update(d)

        return user_sets

    def get_default_parameters(self, user_sets):
        user_params = {
            'S': [{'index': [p], 'value': 0} for p in user_sets['P']],
            'D': [{'index': [c], 'value': 0} for c in user_sets['C']],
            'U': [{'index': [p, c], 'value': 0}
                  for p in user_sets['P'] for c in user_sets['C']],
        }

        return user_params


In [11]:
spec = AIMMSExampleSpecification()

The `mola.build` module allow us to build a concrete model from the `Specification` object using JSON files that contains the sets and parameters that we defined earlier.

First we write the sets to a JSON file.

In [12]:
import json
sets = {'P': list(model.P), 'C':list(model.C)}
sets_json_file = 'Configuration/AIMMS_sets.json'
with open(sets_json_file, 'w') as fp:
    json.dump(sets, fp, indent=4)
sets

{'P': ['Haarlem', 'Eindhoven'],
 'C': ['Amsterdam', 'Breda', 'Gouda', 'Amersfoort', 'Den Bosch']}

Indexed parameter data on disk is stored in index-value form in JSON files. So we convert the parameter data to this form below.

In [13]:
p = spec.get_default_parameters(sets)
for item in p['S']:
    item['value'] = s_data[item['index'][0]]
for item in p['D']:
    item['value'] = d_data[item['index'][0]]
for item in p['U']:
    item['value'] = cost_df.loc[item['index'][0], item['index'][1]]

These parameters are used to overwrite defaults in the specification. The parameters are then written to a JSON file.

In [14]:
import mola.build as mb
import mola.utils as mu
p = mb.build_parameters(sets, p, spec)
parameters = mu.get_index_value_parameters(p)
parameters_json_file = 'Configuration/AIMMS_parameters.json'
with open(parameters_json_file, 'w') as fp: json.dump(parameters, fp, indent=4)

Supply available at plant p
Demand required by customer c
Cost per unit


The two JSON files are used to populate the abstract model in the specification object and generate a new concrete model.

In [15]:
import pyomo.dataportal as pyod
model2 = spec.populate([sets_json_file, parameters_json_file])

Finally, we solve this model using Pyomo and show the delivery plan.

In [16]:
results = opt.solve(model2)
model2.x.pprint()

x : Units
    Size=10, Index=x_index
    Key                         : Lower : Value : Upper : Fixed : Stale : Domain
    ('Eindhoven', 'Amersfoort') :     0 :  31.0 :  None : False : False : NonNegativeReals
     ('Eindhoven', 'Amsterdam') :     0 :   0.0 :  None : False : False : NonNegativeReals
         ('Eindhoven', 'Breda') :     0 :  16.0 :  None : False : False : NonNegativeReals
     ('Eindhoven', 'Den Bosch') :     0 :  12.0 :  None : False : False : NonNegativeReals
         ('Eindhoven', 'Gouda') :     0 :   3.0 :  None : False : False : NonNegativeReals
      ('Haarlem', 'Amersfoort') :     0 :   0.0 :  None : False : False : NonNegativeReals
       ('Haarlem', 'Amsterdam') :     0 :  28.0 :  None : False : False : NonNegativeReals
           ('Haarlem', 'Breda') :     0 :   0.0 :  None : False : False : NonNegativeReals
       ('Haarlem', 'Den Bosch') :     0 :   0.0 :  None : False : False : NonNegativeReals
           ('Haarlem', 'Gouda') :     0 :  19.0 :  None : False

Thus, we show how the model in the specification leads to the same solution as before.